![logoups](https://seeklogo.com/images/U/ups-politecnica-salesiana-logo-C5BE1FA0B6-seeklogo.com.png)

# ANÁLISIS MULTIVARIADO Y MÉTODOS ESTOCÁSTICOS
## Autor: Diego Tapia
## Practica 3: : Aplicaciones de Procesos Estocásticos y Cadenas de Markov



### 1. Resumen: Generación de palabras y textos

En este ejercicio de machine learning, se utiliza el modelo de cadenas de Markov para generar frases nuevas basadas en frases de entrada. Se emplean las frases del siguiente enlace: https://www.lasexta.com/noticias/se-habla/mejores-chistes_202008035f2821076cf6da000167ab56.html


#### Cadenas de Markov
Las cadenas de Markov es un modelo probabilístico que impone que la probabilidad de que suceda algo solo depende del estado anterior. En el contexto de esta aplicacion: la probabilidad de que una palabra sea la siguiente de la frase solo depende de la palabra anterior.


#### Obtención de los datos y Generacion del grafo de Markov

Se obtienen datos del enlace mencionado anteriormente utilizando requests y BeautifulSoup4, y se genera el grafo de Markov mediante un diccionario en Python, utilizando defaultdict para simplificar el código.


In [ ]:
# Obtencion de datos
from bs4 import BeautifulSoup
import requests
print('Módulos importados')

r = requests.get("https://www.lasexta.com/noticias/se-habla/mejores-chistes_202008035f2821076cf6da000167ab56.html")
soup = BeautifulSoup(r.text)

frases = map(lambda x: x.text.replace("\"",""),soup.select("p"))
frases = map(lambda x: x.replace("-",""),frases)
frases = map(lambda x: x.replace("\t",""),frases)
palabras = map(lambda x: str(x).split(" "),frases)
print(palabras)

Módulos importados


In [ ]:
# Generacion del grafo de markov

from collections import defaultdict
almacen = defaultdict(lambda: list())

def add_word(prev,next):
    global almacen
    almacen[prev].append(next)

for frase in palabras:
    frase = list(frase)
    #print(frase)
    for i,palabra in enumerate(frase):
        if i == 0:
            add_word(("START","START"),palabra)
            continue
        if i == 1:
            add_word(("START",frase[0]),palabra)
            continue
        add_word((frase[i-2],frase[i-1]),palabra)

print(almacen)


#### Genreación de Nuevas Frases

Finalmente, se muestra el proceso de generación de una frase nueva utilizando random.choice, y se presentan los resultados.

In [ ]:
#Generando una frase nueva
import random

#Función para generar nueva frase
def gen_word():
    word = list()
    state = "START","START"
    while True:
        w = random.choice(almacen[state])
        word.append(w)
        state = state[1],w
        if w.endswith(".") or w.endswith("!") or w.endswith("\n"):
            return " ".join(word)

In [ ]:
# Generando frases

for a in range(40):
    try:
        palabraGenerada=gen_word()
        print(a,' ',palabraGenerada)
    except Exception as err:
        continue

7    ¿A mí? Infinito.
10   ¡Mira, Iñaki, un Rolex!
12    Pues nada, he ido a comprar 300 vacas, 500 bueyes, 400 ovejas y 30 toneladas de madera.
14   Recopilación de los tuyos, y créeme, hay mucho que reclamar también.
15    ¡AAAHHHHhh!.
18   – Mamá, mamá, los spaghetti se están pegando.
22   – Sonángulos son las personas que caminan dormidas.
25   ¡Rápido, necesitamos sangre!
31   – Porque en el bolsillo.
34    Luke, yo soy tu padre.
35   – Sonángulos son las personas que caminan dormidas.


### 2. Resumen: Reconocimiento de voz

#### Descripción general del reconocimiento de voz

El reconocimiento de voz tiene sus raíces en la investigación realizada en Bell Labs en la década de 1950. Los sistemas modernos han evolucionado significativamente, siendo capaces de reconocer el habla de varios hablantes y tener vocabularios extensos en diversos idiomas. En la actualidad, muchos sistemas utilizan Modelos Ocultos de Markov (HMM, por sus siglas en inglés) para transcribir el audio a texto.

Los HMM dividen la señal de habla en fragmentos de 10 milisegundos, representando el espectro de potencia de cada fragmento con coeficientes cepstrales. Estos coeficientes se utilizan para decodificar la señal en grupos de vectores que se corresponden con fonemas, unidades fundamentales del habla. Los sistemas modernos a menudo emplean redes neuronales para simplificar la señal de habla antes de la reconocimiento HMM, utilizando técnicas de transformación de características y reducción de dimensionalidad. Además, se emplean detectores de actividad vocal para reducir la señal de audio a partes probablemente habladas, optimizando así el proceso.

Afortunadamente, en Python se pueden aprovechar servicios de reconocimiento de voz en línea a través de API, sin preocuparse por los detalles técnicos.

#### Selección del paquete de Reconocimiento de voz

Existen varias opciones de paquetes para reconocimiento de voz en Python disponibles en PyPI, como apiai, assemblyai, google-cloud-speech, pocketsphinx, SpeechRecognition, watson-developer-cloud y wit. Entre ellas, SpeechRecognition destaca por su facilidad de uso y flexibilidad, actuando como un contenedor para varias API populares de reconocimiento de voz. Este paquete, que simplifica la obtención de entrada de audio, es especialmente recomendado por su rapidez en poner en marcha el reconocimiento de voz, evitando la necesidad de construir scripts desde cero para acceder a micrófonos y procesar archivos de audio.


#### Aplicación en python para reconocimiento de voz con SpeechRecognition

In [ ]:
#%pip install SpeechRecognition
#%pip install pyaudio


In [ ]:
import random
import time

import speech_recognition as sr


def recognize_speech_from_mic(recognizer, microphone):
    """Transcribe speech from recorded from `microphone`.

    Returns a dictionary with three keys:
    "success": a boolean indicating whether or not the API request was
               successful
    "error":   `None` if no error occured, otherwise a string containing
               an error message if the API could not be reached or
               speech was unrecognizable
    "transcription": `None` if speech could not be transcribed,
               otherwise a string containing the transcribed text
    """
    # check that recognizer and microphone arguments are appropriate type
    if not isinstance(recognizer, sr.Recognizer):
        raise TypeError("`recognizer` must be `Recognizer` instance")

    if not isinstance(microphone, sr.Microphone):
        raise TypeError("`microphone` must be `Microphone` instance")

    # adjust the recognizer sensitivity to ambient noise and record audio
    # from the microphone
    with microphone as source:
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    # set up the response object
    response = {
        "success": True,
        "error": None,
        "transcription": None
    }

    # try recognizing the speech in the recording
    # if a RequestError or UnknownValueError exception is caught,
    #     update the response object accordingly
    try:
        response["transcription"] = recognizer.recognize_google(audio)
    except sr.RequestError:
        # API was unreachable or unresponsive
        response["success"] = False
        response["error"] = "API unavailable"
    except sr.UnknownValueError:
        # speech was unintelligible
        response["error"] = "Unable to recognize speech"

    return response


if __name__ == "__main__":
    # set the list of words, maxnumber of guesses, and prompt limit
    WORDS = ["apple", "banana", "grape", "orange", "mango", "lemon"]
    NUM_GUESSES = 3
    PROMPT_LIMIT = 5

    # create recognizer and mic instances
    recognizer = sr.Recognizer()
    microphone = sr.Microphone()

    # get a random word from the list
    word = random.choice(WORDS)

    # format the instructions string
    instructions = (
        "I'm thinking of one of these words:\n"
        "{words}\n"
        "You have {n} tries to guess which one.\n"
    ).format(words=', '.join(WORDS), n=NUM_GUESSES)

    # show instructions and wait 3 seconds before starting the game
    print(instructions)
    time.sleep(3)

    for i in range(NUM_GUESSES):
        # get the guess from the user
        # if a transcription is returned, break out of the loop and
        #     continue
        # if no transcription returned and API request failed, break
        #     loop and continue
        # if API request succeeded but no transcription was returned,
        #     re-prompt the user to say their guess again. Do this up
        #     to PROMPT_LIMIT times
        for j in range(PROMPT_LIMIT):
            print('Guess {}. Speak!'.format(i+1))
            guess = recognize_speech_from_mic(recognizer, microphone)
            if guess["transcription"]:
                break
            if not guess["success"]:
                break
            print("I didn't catch that. What did you say?\n")

        # if there was an error, stop the game
        if guess["error"]:
            print("ERROR: {}".format(guess["error"]))
            break

        # show the user the transcription
        print("You said: {}".format(guess["transcription"]))

        # determine if guess is correct and if any attempts remain
        guess_is_correct = guess["transcription"].lower() == word.lower()
        user_has_more_attempts = i < NUM_GUESSES - 1

        # determine if the user has won the game
        # if not, repeat the loop if user has more attempts
        # if no attempts left, the user loses the game
        if guess_is_correct:
            print("Correct! You win!".format(word))
            break
        elif user_has_more_attempts:
            print("Incorrect. Try again.\n")
        else:
            print("Sorry, you lose!\nI was thinking of '{}'.".format(word))
            break

I'm thinking of one of these words:
apple, banana, grape, orange, mango, lemon
You have 3 tries to guess which one.

Guess 1. Speak!
You said: Apple
Incorrect. Try again.

Guess 2. Speak!
You said: Orange
Incorrect. Try again.

Guess 3. Speak!
I didn't catch that. What did you say?

Guess 3. Speak!
I didn't catch that. What did you say?

Guess 3. Speak!
You said: lemon
Sorry, you lose!
I was thinking of 'mango'.


### 3. Aplicación de Cadena de Markov en Python

#### Explicación inicial

Se tiene el siguiente ejemplo de aplicación.

En una Unidad de Cuidados Intensivos en un determinado hospital, cada paciente es clasificado de acuerdo a un estado crítico, serio o estable.

Detalle de los estados:

CRITICO: Este estado podría representar a pacientes que enfrentan situaciones de salud muy complicadas y que tienen una probabilidad considerable de empeorar, aunque aún existe una posibilidad de mejoría.

SERIO: Este estado podría representar a pacientes que están en una situación de salud seria pero no tan crítica como aquellos en estado "CRITICO". La probabilidad de mejoría es un poco más alta, pero también hay una probabilidad considerable de empeorar.

ESTABLE: Un paciente se considera estable cuando sus signos vitales están dentro de los rangos normales, y no experimenta cambios sustanciales en su estado de salud. En otras palabras, no hay indicadores evidentes de empeoramiento o mejora significativa.


Estos estados son actualizadas cada mañana por un médico, de acuerdo a la evaluación experimentada por el paciente.


#### Diseño y Desarrollo

Las probabilidades con las cuales cada paciente pasa de un estado a otro se resumen en siguiente tabla:

#### Tabla 1: Matriz de transición de estados de pacientes
|            | Crítico| Serio | Estable |
|------------|-------------|-----------|-------------|
| **Crítico**  | 0.60        | 0.30      | 0.10        |
| **Serio**    | 0.40        | 0.40      | 0.20        |
| **Estable**  | 0.10        | 0.40      | 0.50        |

#### Cadena de Markov

A partir de la matriz de transición se tiene la siguiente  cadena de markov

Figura 1: Cadena de Markov: Ejemplo Unidad de ciudados Intensivos
![cadenamarkov](https://raw.githubusercontent.com/juandtap/practica_3_cadenas_markov/master/gradomarkov.png)

#### Código en Python

Simulación de cambios de estado de los pacientes en los siguientes 7 dias

In [2]:
import numpy as np

# Definir estados
estados_paciente = ["CRITICO", "SERIO", "ESTABLE"]
# índices estados
indice_estado_paciente = {estado: i for i, estado in enumerate(estados_paciente)}

# Matriz de transición de estados de pacientes
transicion_estados_paciente = np.array([
    # C    S    E
    [0.6, 0.3, 0.1],  # Critico
    [0.4, 0.4, 0.2],  # Serio
    [0.1, 0.4, 0.5],  # Estable
])

# Estado inicial (puede ser aleatorio o definido)
estado_actual_paciente = "CRITICO"

# Simulación de transiciones durante 7 días
num_dias = 7

for dia in range(1, num_dias + 1):
    # Obtener el índice del estado actual
    indice_actual_paciente = indice_estado_paciente[estado_actual_paciente]

    # Elegir el próximo estado según la matriz de transición
    eleccion_paciente = np.random.multinomial(1, transicion_estados_paciente[indice_actual_paciente])

    # Obtener el índice del próximo estado
    indice_proximo_estado_paciente = np.argmax(eleccion_paciente)

    # Obtener el nombre del próximo estado
    proximo_estado_paciente = estados_paciente[indice_proximo_estado_paciente]

    # Actualizar el estado actual para el próximo día
    estado_actual_paciente = proximo_estado_paciente

    # Mostrar el estado del paciente en cada día
    print(f"Día {dia}: Paciente en estado {estado_actual_paciente}")

print("Simulación de 7 días terminada")

Día 1: Paciente en estado CRITICO
Día 2: Paciente en estado CRITICO
Día 3: Paciente en estado SERIO
Día 4: Paciente en estado SERIO
Día 5: Paciente en estado ESTABLE
Día 6: Paciente en estado SERIO
Día 7: Paciente en estado ESTABLE
Simulación de 7 días terminada


También se pueden calcular probabilidades más específicas como:

La probabilidad de que un paciente esté en estado crítico el día Jueves y que el día Sábado esté estable, esta dado por: $\mathbb{P}_{CE}^{2}$, es decir, la probabilidad de pasar del estado crítico al estado estable al cabo de 2 etapas (días).

$\mathbb{P}_{CE}^{2}=0,3*0,2+0,1*0,5+0,6*0,1=0,17$

Tambien se puede usar de forma equivalente las ecuaciones matriciales

$f^{n}=P^{T}*f^{n-1}$:


$f^{1}=P^{T}*f^{0}=$
$
\begin{bmatrix}
    0.60 & 0.40 & 0.10 \\
    0.30 & 0.40 & 0.40 \\
    0.20 & 0.20 & 0.50 \\
\end{bmatrix}$
$
\begin{bmatrix}
    1 \\
    0 \\
    0 \\
\end{bmatrix}
=$
$
\begin{bmatrix}
    0.60 \\
    0.30 \\
    0.10 \\
\end{bmatrix}
$

$f^{2}=P^{T}*f^{1}=$
$
\begin{bmatrix}
    0.60 & 0.40 & 0.10 \\
    0.30 & 0.40 & 0.40 \\
    0.20 & 0.20 & 0.50 \\
\end{bmatrix}
$
$
\begin{bmatrix}
    0.6 \\
    0.3 \\
    0.1\\
\end{bmatrix}
=$
$
\begin{bmatrix}
    0.49 \\
    0.34 \\
    0.17 \\
\end{bmatrix}
$

Se comprueba que la probabilidad de pasar del estado crítico al estado estable al cabo de 2 etapas es de un 17%.

Calculo de probabilidad en python:



In [3]:
# Calcular la probabilidad de pasar de estado CRITICO a ESTABLE dentro de 2 dias (etapas)
pce2 = np.linalg.matrix_power(transicion_estados_paciente, 2)[0][2]

print(f"La probabilidad de pasar de critico  a estable dentro de 2 dias es: {pce2}")

La probabilidad de pasar de critico  a estable dentro de 2 dias es: 0.16999999999999998


¿Cuál es la probabilidad que un paciente que está en estado estable el Lunes experimente alguna complicación y no esté estable nuevamente el Miércoles?.

In [4]:
# Calcular la probabilidad de pasar de estado ESTABLE a CRITICO o SERIO  dentro de 2 dias (etapas)
pecs2 = np.linalg.matrix_power(transicion_estados_paciente, 2)
print(pecs2)
print(f"La probabilidad de pasar estable a critico dentro de 2 dias es: {pecs2[2][0]}")
print(f"La probabilidad de pasar estable a serio dentro de 2 dias es: {pecs2[2][1]}")
print("La probabilidad que un paciente que está en estado estable el Lunes experimente alguna complicación y no esté estable nuevamente el Miércoles es:")
print(f"{pecs2[2][0]} + {pecs2[2][1]} = {pecs2[2][0]+pecs2[2][1]}")


[[0.49 0.34 0.17]
 [0.42 0.36 0.22]
 [0.27 0.39 0.34]]
La probabilidad de pasar estable a critico dentro de 2 dias es: 0.27
La probabilidad de pasar estable a serio dentro de 2 dias es: 0.39
La probabilidad que un paciente que está en estado estable el Lunes experimente alguna complicación y no esté estable nuevamente el Miércoles es:
0.27 + 0.39 = 0.66



#### Conclusiones

Este modelo proporciona una herramienta para analizar y prever la evolución de la condición de los pacientes a lo largo del tiempo (dias).

La simulación de la transición de estados con un vector inicial nos permite comprender cómo las probabilidades influyen en la distribución de pacientes en diferentes estados. En este caso, observamos que al comenzar con un paciente en estado crítico, hay una alta probabilidad de que permanezca crítico en el siguiente paso, indicando la gravedad de este estado.

La aplicación de cadenas de Markov en el ámbito médico facilita el analisis de estados de los pacientes, brindando una visión cuantitativa (probabilidades) que puede ser valiosa a la hora de tomar decisiones con respecto a los pacientes y la gestión de recursos en entornos de cuidados intensivos.

#### Referencias

Arroyo, A. (2018, January 25). generar frases  con cadenas-markov machine learning en python. Adrianistan Ell Blog De Adrian Arroyo. Retrieved January 21, 2024, from https://blog.adrianistan.eu/generar-frases-cadenas-markov-machine-learning-python

Python, R. (2023, September 7). The Ultimate guide to speech recognition with Python. https://realpython.com/python-speech-recognition/

Tutoriales, G. (2015, October 7). Cadenas de Markov (Ejercicios Resueltos). Gestión De Operaciones. https://www.gestiondeoperaciones.net/cadenas-de-markov/cadenas-de-markov-ejercicios-resueltos/